In [2]:
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodadosdestino = "MERCADO_DATAWAREHOUSE"
driver = "ODBC Driver 17 for SQL Server"

conexao_destino = f"mssql+pyodbc://{server}/{bancodadosdestino}?driver={driver}&Trusted_Connection=yes"
engine_destino = create_engine(conexao_destino)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print(f"Query teste executada com sucesso, conexão com o banco de dados {engine} bem-sucedida")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão

testar_conexao_sqlalchemy(engine_destino)

data_inicial = datetime(2000,1,1)
data_final = datetime(2050,12,31)

intervalo_datas = pd.date_range(start=data_inicial, end=data_final, freq='D')

dias_semana_ptbr = {
    'Monday': 'Segunda',
    'Tuesday': 'Terça',
    'Wednesday': 'Quarta',
    'Thursday': 'Quinta',
    'Friday': 'Sexta',
    'Saturday': 'Sabado',
    'Sunday': 'Domingo'
}

meses_ptbr = {
    'January':    'Janeiro',
    'February':   'Fevereiro',
    'March':      'Marco',
    'April':      'Abril',
    'May':        'Maio',
    'June':       'Junho',
    'July':       'Julho',
    'August':     'Agosto',
    'September':  'Setembro',
    'October':    'Outubro',
    'November':   'Novembro',
    'December':   'Dezembro',

}

trimestre_nome = {
    1 : 'Primeiro',
    2 : 'Segundo',
    3 : 'Terceiro',
    4 : 'Quarto',
}

df_interv_datas = pd.DataFrame({'DATA':intervalo_datas,'DIASEMANA':(intervalo_datas.day_name()).map(dias_semana_ptbr)})

def estacao_do_ano(data):
    mes = data.month
    dia = data.day

    if (mes == 12 and dia >= 21) or (mes == 1) or (mes == 2) or (mes == 3 and dia < 21):
        return "Verao"
    elif (mes == 3 and dia >= 21) or (mes == 4) or (mes == 5) or (mes == 6 and dia < 21):
        return "Outono"
    elif (mes == 6 and dia >= 21) or (mes == 7) or (mes == 8) or (mes == 9 and dia < 21):
        return "Inverno"
    elif (mes == 9 and dia >= 21) or (mes == 10) or (mes == 11) or (mes == 12 and dia < 21):
        return "Primavera"
    else:
        return "Data invalida"

df_dim_tempo = pd.DataFrame({
                'DATA':       intervalo_datas,
                'DIA':        intervalo_datas.day,
                'DIASEMANA':  (intervalo_datas.day_name()).map(dias_semana_ptbr),
                'MES':        intervalo_datas.month,
                'NOMEMES':    (intervalo_datas.month_name()).map(meses_ptbr),
                'TRIMESTRE':  intervalo_datas.quarter.map(trimestre_nome),
                'ANO':        intervalo_datas.year,  
                'ESTACAO':    pd.Series(intervalo_datas).apply(estacao_do_ano),
                'FIM_SEMANA': df_interv_datas['DIASEMANA'].isin(['Sabado', 'Domingo']).map({True:'Sim',False:'Nao'}),
                'DATACOMPLETA': intervalo_datas.strftime('%Y%m%d')
})

display(df_dim_tempo)

tipo_dados_DIM_TEMPO ={
                'DATA':         DATE,
                'DIA':          VARCHAR(10),
                'DIASEMANA':    VARCHAR(10),
                'MES':          VARCHAR(2),
                'NOMEMES':      VARCHAR(10),
                'TRIMESTRE':    VARCHAR(10),
                'ANO':          VARCHAR(4),
                'ESTACAO':      VARCHAR(10),
                'FIM_SEMANA':   VARCHAR(3),
                'DATACOMPLETA': VARCHAR(8),
}

try:
    df_dim_tempo.to_sql(name='DIM_TEMPO', con=engine_destino,if_exists='append',index=False,dtype=tipo_dados_DIM_TEMPO)
    print(f"Tabela atualizada com sucesso.")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


Query teste executada com sucesso, conexão com o banco de dados Engine(mssql+pyodbc://DESKTOP-HV85ADA\PROJETO_MERCADO/MERCADO_DATAWAREHOUSE?Trusted_Connection=yes&driver=ODBC+Driver+17+for+SQL+Server) bem-sucedida


,DATA,DIA,DIASEMANA,MES,NOMEMES,TRIMESTRE,ANO,ESTACAO,FIM_SEMANA,DATACOMPLETA
0,2000-01-01,1,Sabado,1,Janeiro,Primeiro,2000,Verao,Sim,20000101
1,2000-01-02,2,Domingo,1,Janeiro,Primeiro,2000,Verao,Sim,20000102
2,2000-01-03,3,Segunda,1,Janeiro,Primeiro,2000,Verao,Nao,20000103
3,2000-01-04,4,Terça,1,Janeiro,Primeiro,2000,Verao,Nao,20000104
4,2000-01-05,5,Quarta,1,Janeiro,Primeiro,2000,Verao,Nao,20000105
...,...,...,...,...,...,...,...,...,...,...
18623,2050-12-27,27,Terça,12,Dezembro,Quarto,2050,Verao,Nao,20501227
18624,2050-12-28,28,Quarta,12,Dezembro,Quarto,2050,Verao,Nao,20501228
18625,2050-12-29,29,Quinta,12,Dezembro,Quarto,2050,Verao,Nao,20501229
18626,2050-12-30,30,Sexta,12,Dezembro,Quarto,2050,Verao,Nao,20501230


Tabela atualizada com sucesso.
